In [18]:
import pandas as pd 
import numpy as np 
import re
from openai import OpenAI
import ast
from tqdm import tqdm 
import matplotlib.pyplot as plt 
import plotly.express as px
import statsmodels.formula.api as smf
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr, pearsonr
import pickle
import itertools


In [19]:
client = OpenAI(api_key=open("/Users/abdullahalibrahim/Desktop/openai_api.txt", "r").read().strip())
client

In [20]:
system_prompt = """Many experimental studies of team performance focus on specific tasks, raising questions about how findings generalize to other tasks.
The lack of commensurability among tasks impedes generalization, as existing taxonomies don't comprehensively, consistently, and quantitatively measure task similarity. 
To address this, create a set of dimensions that classify tasks and estimate similarities by calculating a between-task distance.

For each dimension, provide a quantitative measure to operationalize it.
List the dimensions and their measures, aiming for thorough differentiation between tasks. 
Ensure the dimensions are varied and quantifiable, facilitating meaningful comparisons across diverse tasks.
Use this CSV format: {Dimension; Measure; [DIMENSION 1]; [MEASURE 1]; [DIMENSION 2]; [MEASURE 2]; ...}


"""


response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
        {"role": "system", "content": system_prompt}
        ],
    temperature=0)


In [21]:
response

ChatCompletion(id='chatcmpl-8hkBsOPh0ulDanuVxm600GlQKxg3T', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{Dimension; Measure; Complexity; Number of distinct steps or elements; Physical Demand; Metabolic equivalents (METs) required; Cognitive Load; Subjective mental workload rating (e.g., NASA-TLX score); Team Size; Number of team members; Coordination Requirement; Coordination Load Index (CLI); Task Duration; Time to complete the task in minutes; Novelty; Likert scale rating from familiar (1) to novel (7); Interdependence; Index of Task Interdependence (ITI); Goal Clarity; Likert scale rating from ambiguous (1) to clear (7); Autonomy; Degree of control over task execution (low to high); Feedback Frequency; Number of feedback instances per unit time; Skill Variety; Number of different skills required; Problem-Solving Demand; Heuristic complexity score (low to high); Information Processing; Bits of information processed per unit tim

In [22]:
response.choices[0].message.content

'{Dimension; Measure; Complexity; Number of distinct steps or elements; Physical Demand; Metabolic equivalents (METs) required; Cognitive Load; Subjective mental workload rating (e.g., NASA-TLX score); Team Size; Number of team members; Coordination Requirement; Coordination Load Index (CLI); Task Duration; Time to complete the task in minutes; Novelty; Likert scale rating from familiar (1) to novel (7); Interdependence; Index of Task Interdependence (ITI); Goal Clarity; Likert scale rating from ambiguous (1) to clear (7); Autonomy; Degree of control over task execution (low to high); Feedback Frequency; Number of feedback instances per unit time; Skill Variety; Number of different skills required; Problem-Solving Demand; Heuristic complexity score (low to high); Information Processing; Bits of information processed per unit time; Risk Level; Probability of failure times the consequence of failure; Creativity Requirement; Creativity Rating Scale (CRS) score; Technological Mediation; Le

In [23]:
# Splitting the string into segments
segments = response.choices[0].message.content.split("; ")

# Creating pairs of Dimension and Measure
table_data = []
i = 0
while i < len(segments):
    if i + 1 < len(segments):
        dimension = segments[i].strip()
        measure = segments[i + 1].strip()

        # Avoiding misalignment by checking for 'Scale from' in the next segment
        if i + 2 < len(segments) and segments[i + 2].startswith("Scale from"):
            measure += "; " + segments[i + 2].strip()
            i += 1  # Skip the next segment as it's part of the current measure

        table_data.append((dimension, measure))
    i += 2

# Create DataFrame
df_refined = pd.DataFrame(table_data, columns=['Dimension', 'Measure'])
df = df_refined.drop(df_refined.index[0])
df

,Dimension,Measure
1,Complexity,Number of distinct steps or elements
2,Physical Demand,Metabolic equivalents (METs) required
3,Cognitive Load,"Subjective mental workload rating (e.g., NASA-..."
4,Team Size,Number of team members
5,Coordination Requirement,Coordination Load Index (CLI)
6,Task Duration,Time to complete the task in minutes
7,Novelty,Likert scale rating from familiar (1) to novel...
8,Interdependence,Index of Task Interdependence (ITI)
9,Goal Clarity,Likert scale rating from ambiguous (1) to clea...
10,Autonomy,Degree of control over task execution (low to ...


In [24]:
df.to_csv('DirectAskOutput.csv', index=False)